In [18]:
!pip install pytesseract


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
!pip install pytesseract


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [20]:
!pip install pdf2image


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
import os
import requests
import pdfplumber
import pytesseract
from pdf2image import convert_from_path

def extract_text_from_pdf(pdf_path):
    """Try extracting text using pdfplumber, fallback to OCR if needed."""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            text = "\n".join([p.extract_text() or "" for p in pdf.pages]).strip()
        if text:
            return text
    except Exception as e:
        print("pdfplumber failed:", e)

    print("Using OCR fallback...")
    images = convert_from_path(pdf_path)
    ocr_text = ""
    for img in images:
        ocr_text += pytesseract.image_to_string(img)
    return ocr_text.strip()

def ask_pdf_question(pdf_text, question, api_url="http://localhost:1234/v1/chat/completions", model_name="phi-3.1-mini-128k-instruct"):
    """Sends the PDF content and user question to the model."""
    prompt = f"You are given the following document:\n\n{pdf_text[:12000]}\n\nAnswer the following question based on it:\n\n{question}"

    payload = {
        "model": model_name,
        "messages": [
            {"role": "system", "content": "You are a helpful assistant that answers questions based on provided documents."},
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.3,
        "max_tokens": 1024
    }

    response = requests.post(api_url, json=payload)
    if response.status_code == 200:
        return response.json()['choices'][0]['message']['content']
    else:
        raise Exception(f"API Error {response.status_code}: {response.text}")


In [ ]:

if __name__ == "__main__":
    pdf_path = input("Enter the path to your PDF file: ").strip()

    if not os.path.exists(pdf_path):
        print("PDF file not found.")
        exit(1)

    print("Extracting text from PDF...")
    pdf_text = extract_text_from_pdf(pdf_path)

    if not pdf_text:
        print("No text could be extracted.")
        exit(1)

    print("\nYou can now ask questions about the PDF.\nType 'exit' to quit.")

    while True:
        question = input("\nAsk a question: ").strip()
        if question.lower() in ["exit", "quit"]:
            break
        try:
            answer = ask_pdf_question(pdf_text, question)
            print("\nAnswer:\n", answer)
        except Exception as e:
            print("Error:", e)


Enter the path to your PDF file:  TataComms-q4-fy2025-1.pdf


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


Extracting text from PDF...


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox



You can now ask questions about the PDF.
Type 'exit' to quit.



Ask a question:  who is Mr. Kabir Ahmed Shakir



Answer:
 Mr. Kabir Ahmed Shakir is the CFO of Tata Communications, as mentioned in the document where Sudeshna Patnaik introduces him during an earnings call for Q4 & full-year FY2025 Earnings Call. He provides financial highlights and insights into the company's performance after Amur Lakshminarayanan discusses business outlook, growth in various segments, and strategic interventions made during FY25.
